# Feature extraction

## Module imports

In [1]:
import numpy as np
import os
from pathlib import Path
import pickle
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

import sys
sys.path.append('..')
from helper.classification_tools import CustomLabelEncoder

## Loading files

In [2]:
img_root = Path('..','data','images_preprocessed','images_histeq_resize')  # directory where images are stored.
assert img_root.is_dir() # make sure directory exists and is properly found
files = sorted(img_root.glob("*.bmp"))  # returns a list of all of the images in the directory, sorted by filename.

## Shuffle the filenames so they appear randomly in the dataset. 
rs = np.random.RandomState(seed=749976)
rs.shuffle(files)

assert len(files) == 1800  # make sure all files are found.
print('first 10 filenames: {}'.format([x.name for x in files[:10]]))

first 10 filenames: ['Pa_286.bmp', 'RS_126.bmp', 'Sc_295.bmp', 'In_20.bmp', 'Cr_79.bmp', 'Cr_40.bmp', 'RS_214.bmp', 'In_180.bmp', 'Cr_274.bmp', 'RS_108.bmp']


## Extracting the labels from filenames

In [3]:
def extract_labels(f): return [x.stem.split('_')[0] for x in f]
labels = extract_labels(files)
print('first 10 labels: {}'.format(labels[:10]))

first 10 labels: ['Pa', 'RS', 'Sc', 'In', 'Cr', 'Cr', 'RS', 'In', 'Cr', 'RS']


## Label encoding

In [4]:
le = CustomLabelEncoder()
le.fit(labels, sorter=lambda x: x.upper())

labels_int = le.transform(labels[:10])
labels_str = le.inverse_transform(labels_int)


with open(Path('..','models','label_encoder.pickle'), 'wb') as f:
    pickle.dump(le, f)

print('label encodings: {}'.format(le.mapper))
print('first 10 integer labels: {}'.format(labels_int))
print('first 10 string labels: {}'.format(labels_str))

label encodings: {'Cr': 0, 'In': 1, 'Pa': 2, 'PS': 3, 'RS': 4, 'Sc': 5}
first 10 integer labels: [2 4 5 1 0 0 4 1 0 4]
first 10 string labels: ['Pa' 'RS' 'Sc' 'In' 'Cr' 'Cr' 'RS' 'In' 'Cr' 'RS']


## Loading Images

In [5]:
def load_images(paths):
    """
    Loads images in the correct format for use with the Keras VGG16 model
    
    Images are loaded as PIL image objects, converted to numpy array, and then formatted
    with the appropriate VGG16.preprocess_input() function. Note that this only changes
    how the images are represented, it does not change the actual visual properties of the
    images like preprocessing did before.
    
    Parameters
    ----------
    paths: list(Path)
        list of Paths to each file where the image is stored. Note that the images should 
        have the same height, width in pixels so they can be stored in one array.
    
    Returns
    ----------
    images: ndarray
        n_images x r x c x 3 array of pixel values that is compatible with the Keras model.
    
    """
    
    images = [image.load_img(file) for file in paths] # load images
    # convert images to an array with shape consistent for the vgg16 input
    images = np.asarray([image.img_to_array(img) for img in images]) 
    # normalizes the pixel values to match the imagenet format (and therefore the pre-trained weights)
    images = preprocess_input(images) 
    
    return images

    

In [6]:
images = load_images(files)
assert len(images) == 1800
print(images.shape)

(1800, 224, 224, 3)


# Feature extraction

In [7]:
vgg16_path = Path('..','models','VGG16.h5')
if not vgg16_path.is_file():
    vgg16 = keras.applications.VGG16(include_top=True,  # include fully connected layers
                                     weights='imagenet') # use pre-trained model
    vgg16.save(vgg16_path) # save model so we don't have to download it everytime
    
else:   
    vgg16 = keras.models.load_model(vgg16_path) # use saved model



In [8]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

The pre-trained model will run data through the entire network and return the output of the classification layer. 

Howevever, we only want the output of the intermediate layer so that we can use it as a feature descriptor. 

By default, we will use the fc1 (first fully-connected layer), but we will use other layers as well in the sensitivity analysis.

In [9]:
def layer_extractor(model=vgg16, layer='fc1'):
    """
    returns a model that will extract the outputs of *layer* from *model*.
    
    Parameters
    -------------
    model: keras model
        full model from which intermediate layer will be extracted
    layer: string
        name of layer from which to extract outputs
    
    Returns
    -------------
    new_model: keras model
        feature extractor model which takes the same inputs as *model* and returns the outputs
        of the intermediate layer specified by *layer* by calling new_model.predict(inputs)
    """
    assert layer in [x.name for x in model.layers]  # make sure the layer exists

    new_model = keras.Model(inputs = vgg16.input, outputs=[vgg16.get_layer(layer).output])
    
    return new_model




## FC1 features

In [10]:
fc1_extractor = layer_extractor()
fc1 = fc1_extractor.predict(images, verbose=True)

# save results
results = {'filename' : files,
           'features': fc1,
          'labels': labels,
           'layer_name': 'fc1'
          }

feature_dir = Path('..','data','features')
os.makedirs(feature_dir, exist_ok=True)
with open(feature_dir / 'VGG16_fc1_features_std.pickle', 'wb') as f:
    pickle.dump(results, f)

print(fc1.shape)

57/57 [==============================] - 156s 3s/step
(1800, 4096)


## Features from other layers

In [11]:
'block3_conv2' in [x.name for x in vgg16.layers]

True

In [12]:
for layer in ['fc2', 'block5_pool', 'block5_conv3']:
    print(f'extracting features: {layer}')
    extractor = layer_extractor(layer=layer)  # model to extract features for each layer
    features = extractor.predict(images, verbose=True)  # features extracted by model
    # save the results using the same format as before
    results = {'filename': files,
              'features': features,
              'labels': labels,
              'layer_name': layer}
    with open(feature_dir / 'VGG16_{}_features.pickle'.format(layer), 'wb') as f:
        pickle.dump(results, f)

extracting features: fc2
57/57 [==============================] - 152s 3s/step
extracting features: block5_pool
57/57 [==============================] - 150s 3s/step
extracting features: block5_conv3
57/57 [==============================] - 150s 3s/step


## FC1 features without histogram equalization

In [13]:
img_root_nohisteq = Path('..','data','images_preprocessed','images_resize')
assert img_root_nohisteq.is_dir()
files_noh = sorted(img_root_nohisteq.glob('*'))
rs = np.random.RandomState(seed=3626210179)
rs.shuffle(files_noh)
labels_noh = extract_labels(files_noh)
assert len(files_noh) == 1800
print('first 5 filenames and labels')
print([x.name for x in files_noh[:5]])
print(labels_noh[:5])

first 5 filenames and labels
['Pa_291.bmp', 'Pa_116.bmp', 'Cr_136.bmp', 'Cr_232.bmp', 'Pa_188.bmp']
['Pa', 'Pa', 'Cr', 'Cr', 'Pa']


In [14]:

images_noh = load_images(files_noh)
fc1_noh = fc1_extractor.predict(images_noh, verbose=True)

results = {'filename': files_noh,
          'features': fc1_noh,
          'labels': labels_noh,
          'layer_name': 'fc1 no_histeq'}
with open(feature_dir / 'VGG16_fc1_features_NoHistEQ.pickle', 'wb') as f:
    pickle.dump(results, f)



57/57 [==============================] - 153s 3s/step
